In [12]:
import pandas as pd
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer

In [21]:
df = pd.read_csv("data/ACS_50k_RACE_3.csv", index_col="id")

In [22]:
# Create metadata for the dataframe
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=df)

# Create an instance of the CTGANSynthesizer with the metadata
synthesizer = CTGANSynthesizer(metadata)

# Fit the synthesizer to the data
synthesizer.fit(df)

# Generate synthetic data samples
synthetic_data = synthesizer.sample(364)  # Generate 364 synthetic samples

synthetic_data.head()


/Users/maxschaffelder/venv/first_venv/lib/python3.10/site-packages/sdv/single_table/base.py:84: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,ST,PINCP
0,27.0,1.0,17.0,1.0,2990.0,45.0,0.0,41.0,2.0,3.0,46.0,240.0
1,26.0,7.0,24.0,1.0,10.0,40.0,0.0,38.0,1.0,3.0,16.0,11448.0
2,33.0,1.0,19.0,4.0,4193.0,20.0,2.0,39.0,1.0,3.0,41.0,47577.0
3,48.0,1.0,18.0,5.0,10.0,20.0,0.0,23.0,1.0,3.0,6.0,16624.0
4,45.0,1.0,17.0,3.0,2986.0,6.0,0.0,41.0,2.0,3.0,56.0,240.0


In [23]:
#synthetic_data.to_csv("data/CTGAN_data.csv")